In [10]:
from time import sleep
from tasho.templates.P2P import P2P
from tasho import TaskModel, default_mpc_options
from tasho.templates.Regularization import Regularization
from tasho.Variable import Variable
from robotshyu import Robot as rob
from tasho.OCPGenerator import OCPGenerator
import numpy as np
import casadi as cs
import tasho
import robotshyu
from tasho.ConstraintExpression import ConstraintExpression
from tasho.Expression import Expression
from tasho.MPC import MPC 
from tasho import environment as env
from tasho import WorldSimulator
import pybullet as p

# Ex1) P2P example

In [11]:
robot = rob.Robot("indy7")
link_name = 6
goal_pose = Variable(robot.name, "goal_pose", "magic_number", (4,4), np.array(
        [[0, 0, 1, 1], [0, 1, 0, -0.1], [-1, 0, 0, 0.6], [0, 0, 0, 1]]
    ))
q0 = [1.0, -0.05311582280659044, -0.815452580946695, -1.3191046402052224, -0.8582660722530533, 1.3988994390898029]

robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)
ndof = robot.nq
q_current = Variable(robot.name, "jointpos_init", 'magic_number', (ndof, 1), q0)

Loading robot params from json ...
Loaded 6-DoF robot: indy7


In [12]:
task_P2P = P2P(robot, link_name, goal_pose, q_current, 0.001)

In [13]:
task_P2P.write_task_graph("Indy7_P2P_Task.svg")

In [14]:
reg_jacc = task_P2P.constraint_expressions['reg_qdd_'+robot.name].change_weight(0.1)
task_P2P.add_path_constraints(Regularization(task_P2P.variables['qd_'+robot.name], 1))

In [15]:
task_P2P.write_task_graph("Indy7_P2P_Task2.svg")

In [16]:
horizon_steps = 10
horizon_period = 3.0
q_pred = [0]*horizon_steps

OCP_gen = OCPGenerator(task_P2P, False, {"time_period": horizon_period, "horizon_steps":horizon_steps})
q_ocp = OCP_gen.stage_tasks[0].variables['q_'+robot.name].x
OCP_gen.tc.set_initial(q_ocp, q0)
OCP_gen.tc.set_ocp_solver(
    "ipopt", 
    # {"ipopt":{"linear_solver":"ma27"}}
)
OCP_gen.tc.solve_ocp()

This is Ipopt version 3.12.7, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      452
Number of nonzeros in inequality constraint Jacobian.:      186
Number of nonzeros in Lagrangian Hessian.............:      141

Total number of variables............................:      192
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      141
Total number of inequality constraints...............:      181
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:      180
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 6.46e-01 9.16e-02  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1

In [17]:
t_grid, qsol = OCP_gen.tc.sol_sample(q_ocp)
_, q_dot_sol = OCP_gen.tc.sol_sample(OCP_gen.stage_tasks[0].variables['qd_'+robot.name].x)

obj = WorldSimulator.WorldSimulator()

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
indy7_urdf = robotshyu.__path__[0]+"/robots/NRMK/indy7/indy7_description/indy7.urdf"
indy7_frame_urdf = robotshyu.__path__[0]+"/robots/NRMK/indy7/indy7_description/indy7_frame.urdf"

robotID = obj.add_robot(position,orientation,robot_urdf=indy7_urdf)
frameID0 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID1 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID2 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID3 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID4 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID5 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID6 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID7 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID8 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID9 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID10 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)

robotIDs=[robotID,frameID0]
frameIDs=[frameID1,frameID2,frameID3,frameID4,frameID5,frameID6,frameID7,frameID8,frameID9,frameID10]

for i in range(horizon_steps):
    q_pred[i]=qsol[i]

joint_indices = [0, 1, 2, 3, 4, 5]

obj.resetMultiJointState(robotIDs, joint_indices, [q0])
obj.resetMultiJointState(frameIDs, joint_indices, [q0])


obj.resetMultiJointState(frameIDs,joint_indices,q_pred)

True

In [18]:
for i in range(horizon_steps + 1):
    sleep(horizon_period*0.5/horizon_steps)
    obj.resetMultiJointState(
        robotIDs, joint_indices, [qsol[i]]
    )
    
sleep(0.5)
obj.end_simulation()

Ending simulation


# Ex2) Object picking MPC

In [9]:
robot = rob.Robot("indy7")
link_name = 6

# Define initial conditions of the robot
q0_val = [0, -0.423598, -0.9, 0.0, -1.5709, -0.523598]
qd0_val = [0] * robot.ndof

robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)

################################################
# Task spacification - Approximation to object
################################################

# Select prediction horizon and sample time for the MPC execution
horizon_size = 10
t_mpc = 0.05

q_init = Variable(robot.name, 'q_init', 'parameter', (robot.ndof,1))
qd_init = Variable(robot.name, 'qd_init', 'parameter', (robot.ndof,1))
goal_pose = Variable(robot.name, "goal_pose", 'parameter', (4,4))

task_P2P = P2P(robot, link_name, goal_pose, q_init, 0.001)
task_P2P.write_task_graph("Indy7_P2P_Task4.svg")

Loading robot params from json ...
Loaded 6-DoF robot: indy7


In [10]:
task_P2P.remove_initial_constraints(task_P2P.constraint_expressions['stationary_qd_indy7'])
task_P2P.remove_terminal_constraints('rot_con_pose_6_indy7_vs_goal',
                                    'trans_con_pose_6_indy7_vs_goal')
task_P2P.write_task_graph("Indy7_P2P_Task5.svg")

In [11]:
task_P2P.add_initial_constraints(ConstraintExpression(robot.name, "eq", Expression(robot.name, "err_qd_qinit", lambda a, b : a - b, qd_init, task_P2P.variables['qd_indy7']),
                             "hard", reference = 0))
task_P2P.add_path_constraints(Regularization(task_P2P.expressions['trans_error_pose_6_indy7_vs_goal'], 5e0, norm = "L1"),
                            Regularization(task_P2P.expressions['ax_ang_error_pose_6_indy7_vs_goal'], 1e0), 
                            Regularization(task_P2P.variables['qd_indy7'], 1e-1))

task_P2P.write_task_graph("Indy7_P2P_Task6.svg")

In [12]:
pOCP = OCPGenerator(task_P2P, False, {"time_period":horizon_size*t_mpc, "horizon_steps":horizon_size})

tc = pOCP.tc

################################################
# Set solver and discretization options
################################################
tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 5,"tol": 1e-3}})
# tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 0,"tol": 1e-3, "linear_solver":"ma27"}}) #use this if you have hsl


################################################
# Set parameter values
################################################
q = pOCP.stage_tasks[0].variables['q_indy7'].x
qd = pOCP.stage_tasks[0].variables['qd_indy7'].x
q_0 = pOCP.stage_tasks[0].variables['q_init_indy7'].x
qd_0 = pOCP.stage_tasks[0].variables['qd_init_indy7'].x
goal_pose = pOCP.stage_tasks[0].variables['goal_pose_indy7'].x
goal_pose_val =  cs.vertcat(
    cs.hcat([-1, 0, 0, 0.5]),
    cs.hcat([0, 1, 0, 0.0]),
    cs.hcat([0, 0, -1, 0.25]),
    cs.hcat([0, 0, 0, 1]),
)
tc.set_initial(q, q0_val)
tc.set_value(goal_pose, goal_pose_val)
tc.set_value(q_0, q0_val)
tc.set_value(qd_0, qd0_val)

# Add an output port for joint velocities as well
tc.tc_dict["out_ports"].append({"name":"port_out_qd_indy7", "var":"qd_indy7", "desc": "output port for the joint velocities"})

# Add a monitor for termination criteria
tc.add_monitor({"name":"termination_criteria", "expression":cs.sqrt(cs.sumsqr(pOCP.stage_tasks[0].expressions["trans_error_pose_6_indy7_vs_goal"].x)) - 2e-2, "reference":0, "lower":True, "initial":True})

mpc_options = default_mpc_options.get_default_mpc_options()


tc.ocp_solver = "ipopt"
tc.ocp_options = mpc_options["ipopt_hsl"]["options"]
tc.mpc_solver = tc.ocp_solver
tc.mpc_options = tc.ocp_options
tc.set_ocp_solver("ipopt", tc.mpc_options)
import os; os.system("export OMP_NUM_THREADS = 1")

sol = tc.solve_ocp()

      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  68.00us ( 22.67us)  67.39us ( 22.46us)         3
       nlp_g  |  21.00us (  7.00us)  20.90us (  6.97us)         3
    nlp_grad  |  79.00us ( 79.00us)  78.82us ( 78.82us)         1
  nlp_grad_f  | 171.00us ( 57.00us) 172.44us ( 57.48us)         3
  nlp_hess_l  | 627.00us (313.50us) 627.02us (313.51us)         2
   nlp_jac_g  |  26.00us (  8.67us)  26.67us (  8.89us)         3
       total  |   2.03ms (  2.03ms)   2.02ms (  2.02ms)         1


sh: 1: export: : bad variable name


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, ide

In [13]:
vars_db = tc.generate_MPC_component("./", {"ocp_cg_opts":{"save":True, "codegen":False, "jit":False}, "mpc":True, "mpc_cg_opts":{"save":True, "codegen":False, "jit":False}})

@1=(opti1_p_2[12:15]-fk_T(opti1_x_1){6}[12:15]), (sqrt(dot(@1, @1))-0.02)
@1=(opti1_p_2[12:15]-fk_T(opti1_x_1){6}[12:15]), (sqrt(dot(@1, @1))-0.02)


In [14]:
MPC_component = MPC("indy7_obj_pickup", "./" + tc.name + ".json")

In [15]:
obj = WorldSimulator.WorldSimulator(bullet_gui=True)

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
indy7_urdf = robotshyu.__path__[0]+"/robots/NRMK/indy7/indy7_description/indy7.urdf"
indy7_frame_urdf = robotshyu.__path__[0]+"/robots/NRMK/indy7/indy7_description/indy7_frame.urdf"

robotID = obj.add_robot(position,orientation,robot_urdf=indy7_urdf)
frameID0 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID1 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID2 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID3 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID4 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID5 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID6 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID7 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID8 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)
frameID9 = obj.add_robot(position,orientation,robot_urdf=indy7_frame_urdf)

robotIDs=[robotID,frameID0,frameID1,frameID2,frameID3,frameID4,frameID5,frameID6,frameID7,frameID8,frameID9]

# Set environment
environment = env.Environment()
package_path = tasho.__path__[0]
cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small.urdf")
environment.add_object(cube1, "cube")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf =package_path+ "/models/objects/table.urdf")
environment.add_object(table1, "table1")

cube2 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small_col.urdf", fixed = True)
environment.add_object(cube2, "cube2")
environment.set_in_world_simulator(obj)
cubeID = environment.get_object_ID("cube")
cube2ID = environment.get_object_ID("cube2")

p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.8, 0])
# Determine number of samples that the simulation should be executed
no_samples = int(t_mpc / obj.physics_ts)
if no_samples != t_mpc / obj.physics_ts:
    print("[ERROR] MPC sampling time not integer multiple of physics sampling time")

# Correspondence between joint numbers in bullet and OCP
joint_indices = [0, 1, 2, 3, 4, 5]

 # Begin the visualization by applying the initial control signal
ts, q_sol = tc.sol_sample(q, grid="control")
ts, q_dot_sol = tc.sol_sample(qd, grid="control")
obj.resetMultiJointState(robotIDs, joint_indices, [q0_val])

obj.setController(
    robotID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)

12


In [16]:
# Execute the MPC loop
q_log = []
q_dot_log = []
predicted_pos_log = []
q_pred = [0]*horizon_size

for i in range(horizon_size * 30):
    print("----------- MPC execution -----------")

    q_now = obj.readJointPositions(robotID, joint_indices)
    qd_now = obj.readJointVelocities(robotID, joint_indices)
        
    MPC_component.input_ports["port_inp_q_init_indy7"]["val"] = q_now
    MPC_component.input_ports["port_inp_qd_init_indy7"]["val"] = qd_now

    # Predict the position of the target object (cube)
    lin_vel, ang_vel = p.getBaseVelocity(cubeID)
    lin_vel = cs.DM(lin_vel)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)
    time_to_stop = cs.norm_1(lin_vel) / 0.3
    predicted_pos = (
        cs.DM(lin_pos)
        + cs.DM(lin_vel) * time_to_stop
        - 0.5 * 0.5 * lin_vel / (cs.norm_1(lin_vel) + 1e-3) * time_to_stop ** 2
    )
    predicted_pos_log.append(predicted_pos.full())
    p.resetBasePositionAndOrientation(cube2ID, predicted_pos.full(), [0.0, 0.0, 0.0, 1.0])
    predicted_pos[2] += 0.06  # cube height
    print("Predicted position of cube", predicted_pos)
    predicted_pos_val = cs.vertcat(
        cs.hcat([-1, 0, 0, predicted_pos[0]]),
        cs.hcat([0, 1, 0, predicted_pos[1]]),
        cs.hcat([0, 0, -1, predicted_pos[2]]),
        cs.hcat([0, 0, 0, 1]),
    )

    MPC_component.input_ports["port_inp_goal_pose_indy7"]["val"] = cs.vec(predicted_pos_val)
        
    if i == 0:
        MPC_component.configMPC()

    MPC_component.runMPC()
    sol = MPC_component.res_vals
    for i in range(horizon_size):
        q_pred[i]=sol[6*i:6*i+6].full()
    obj.resetMultiJointState(robotIDs[1:], joint_indices, q_pred)
#     print(q_pred)
    q_log.append(q_now)
    q_dot_log.append(qd_now)

    # Set control signal to the simulated robot
    qd_control_sig = MPC_component.output_ports["port_out_qd_indy7"]["val"].full()
    qdd_control_sig = (MPC_component.output_ports["port_out_qdd_indy7"]["val"] * t_mpc).full()
    
    obj.setController(
        robotID, "velocity", joint_indices, targetVelocities=qd_control_sig+qdd_control_sig
    )
    
    # Simulate
    obj.run_simulation(no_samples)

    # Termination criteria
#     if "termination_criteria_true" in MPC_component.event_output_port:
#         break

----------- MPC execution -----------
Predicted position of cube [0.5, 0.15777500346789, 0.41]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 129.00us ( 32.25us) 127.32us ( 31.83us)         4
       nlp_g  |  37.00us (  9.25us)  37.37us (  9.34us)         4
    nlp_grad  |  86.00us ( 86.00us)  86.32us ( 86.32us)         1
  nlp_grad_f  |   8.34ms (  2.08ms) 343.75us ( 85.94us)         4
  nlp_hess_l  |   1.23ms (411.00us)   1.23ms (411.60us)         3
   nlp_jac_g  |  54.00us ( 13.50us)  54.40us ( 13.60us)         4
       total  |  11.99ms ( 11.99ms)   3.98ms (  3.98ms)         1
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context c

----------- MPC execution -----------
Predicted position of cube [0.49999969844135, 0.26095067357299, 0.27272327926623]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 154.00us ( 38.50us) 154.06us ( 38.52us)         4
       nlp_g  |  47.00us ( 11.75us)  45.38us ( 11.35us)         4
    nlp_grad  | 124.00us (124.00us) 124.65us (124.65us)         1
  nlp_grad_f  | 387.00us ( 96.75us) 384.42us ( 96.11us)         4
  nlp_hess_l  |   6.10ms (  2.03ms)   1.61ms (537.39us)         3
   nlp_jac_g  |  48.00us ( 12.00us)  48.71us ( 12.18us)         4
       total  |  21.32ms ( 21.32ms)   4.61ms (  4.61ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999969850281, 0.27296067008903, 0.27257166572131]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 118.00us ( 39.33us) 118.62us ( 39.54us)         3
       nlp_g  |  38.00us ( 12.67us)  37.35us ( 12.45us)         3
    nlp_grad  | 133.00us (133.00us

----------- MPC execution -----------
Predicted position of cube [0.49999827528198, 0.41660040015309, 0.085038743578075]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  91.00us ( 30.33us)  91.07us ( 30.36us)         3
       nlp_g  |  30.00us ( 10.00us)  29.74us (  9.91us)         3
    nlp_grad  | 101.00us (101.00us) 100.84us (100.84us)         1
  nlp_grad_f  | 228.00us ( 76.00us) 227.59us ( 75.86us)         3
  nlp_hess_l  |   2.82ms (  1.41ms) 777.42us (388.71us)         2
   nlp_jac_g  |  41.00us ( 13.67us)  41.96us ( 13.99us)         3
       total  |  13.00ms ( 13.00ms)   2.94ms (  2.94ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999821825101, 0.42093115375574, 0.085004521962702]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  93.00us ( 31.00us)  93.67us ( 31.22us)         3
       nlp_g  |  29.00us (  9.67us)  28.64us (  9.55us)         3
    nlp_grad  | 108.00us (108.00

----------- MPC execution -----------
Predicted position of cube [0.4999976310874, 0.43633000724449, 0.084989999995779]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  94.00us ( 31.33us)  94.62us ( 31.54us)         3
       nlp_g  |  30.00us ( 10.00us)  29.12us (  9.71us)         3
    nlp_grad  | 104.00us (104.00us) 104.08us (104.08us)         1
  nlp_grad_f  | 235.00us ( 78.33us) 235.44us ( 78.48us)         3
  nlp_hess_l  | 804.00us (402.00us) 804.88us (402.44us)         2
   nlp_jac_g  |  41.00us ( 13.67us)  41.36us ( 13.79us)         3
       total  |   2.74ms (  2.74ms)   2.73ms (  2.73ms)         1
----------- MPC execution -----------
Predicted position of cube [0.4999976310865, 0.4363300072436, 0.084989999998441]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 600.00us (200.00us) 104.66us ( 34.89us)         3
       nlp_g  | 657.00us (219.00us)  46.66us ( 15.55us)         3
    nlp_grad  | 102.00us (102.00us)

----------- MPC execution -----------
Predicted position of cube [0.49999763108597, 0.43633000724301, 0.084989999999986]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   8.10ms (  2.70ms)  94.57us ( 31.52us)         3
       nlp_g  |  28.00us (  9.33us)  28.17us (  9.39us)         3
    nlp_grad  | 107.00us (107.00us) 107.21us (107.21us)         1
  nlp_grad_f  | 227.00us ( 75.67us) 228.01us ( 76.00us)         3
  nlp_hess_l  | 805.00us (402.50us) 805.11us (402.55us)         2
   nlp_jac_g  |  41.00us ( 13.67us)  41.11us ( 13.70us)         3
       total  |  10.71ms ( 10.71ms)   2.70ms (  2.70ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999763108597, 0.43633000724301, 0.084989999999986]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 123.00us ( 41.00us) 122.06us ( 40.69us)         3
       nlp_g  |  36.00us ( 12.00us)  35.30us ( 11.77us)         3
    nlp_grad  | 132.00us (132.00

...

----------- MPC execution -----------
Predicted position of cube [0.33585380884066, 0.2939398792727, 0.084989999764954]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  85.00us ( 28.33us)  86.40us ( 28.80us)         3
       nlp_g  |  28.00us (  9.33us)  27.66us (  9.22us)         3
    nlp_grad  | 104.00us (104.00us) 104.22us (104.22us)         1
  nlp_grad_f  | 227.00us ( 75.67us) 227.51us ( 75.84us)         3
  nlp_hess_l  |   8.82ms (  4.41ms) 818.48us (409.24us)         2
   nlp_jac_g  |  40.00us ( 13.33us)  41.41us ( 13.80us)         3
       total  |  10.70ms ( 10.70ms)   2.69ms (  2.69ms)         1
----------- MPC execution -----------
Predicted position of cube [0.33585380884061, 0.2939398792725, 0.084989999764972]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  88.00us ( 29.33us)  88.01us ( 29.34us)         3
       nlp_g  |  28.00us (  9.33us)  28.87us (  9.62us)         3
    nlp_grad  | 101.00us (101.00us

----------- MPC execution -----------
Predicted position of cube [0.33585380877823, 0.29393987894168, 0.084989999766594]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  87.00us ( 29.00us)  87.96us ( 29.32us)         3
       nlp_g  |  28.00us (  9.33us)  27.98us (  9.33us)         3
    nlp_grad  | 101.00us (101.00us) 101.23us (101.23us)         1
  nlp_grad_f  |   8.23ms (  2.74ms) 228.96us ( 76.32us)         3
  nlp_hess_l  | 802.00us (401.00us) 803.50us (401.75us)         2
   nlp_jac_g  |  42.00us ( 14.00us)  41.77us ( 13.92us)         3
       total  |  10.73ms ( 10.73ms)   2.72ms (  2.72ms)         1
----------- MPC execution -----------
Predicted position of cube [0.33585380877823, 0.29393987894168, 0.084989999766597]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 619.00us (206.33us)  90.46us ( 30.15us)         3
       nlp_g  |  31.00us ( 10.33us)  29.49us (  9.83us)         3
    nlp_grad  | 104.00us (104.00

----------- MPC execution -----------
Predicted position of cube [0.33585380871686, 0.29393987862123, 0.084989999769246]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 102.00us ( 34.00us)  88.82us ( 29.61us)         3
       nlp_g  |  34.00us ( 11.33us)  34.79us ( 11.60us)         3
    nlp_grad  |   3.57ms (  3.57ms) 113.66us (113.66us)         1
  nlp_grad_f  | 410.00us (136.67us) 232.82us ( 77.61us)         3
  nlp_hess_l  |   8.35ms (  4.17ms) 813.75us (406.88us)         2
   nlp_jac_g  |  42.00us ( 14.00us)  42.00us ( 14.00us)         3
       total  |  15.63ms ( 15.63ms)   2.78ms (  2.78ms)         1
----------- MPC execution -----------
Predicted position of cube [0.33585380871628, 0.29393987861883, 0.084989999769453]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  88.00us ( 29.33us)  88.98us ( 29.66us)         3
       nlp_g  |  29.00us (  9.67us)  28.75us (  9.58us)         3
    nlp_grad  | 103.00us (103.00

----------- MPC execution -----------
Predicted position of cube [0.33585380871596, 0.29393987861759, 0.084989999769591]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  86.00us ( 28.67us)  85.01us ( 28.34us)         3
       nlp_g  |  36.00us ( 12.00us)  35.05us ( 11.68us)         3
    nlp_grad  |   1.85ms (  1.85ms) 111.09us (111.09us)         1
  nlp_grad_f  |   2.74ms (913.33us) 235.87us ( 78.62us)         3
  nlp_hess_l  | 813.00us (406.50us) 813.32us (406.66us)         2
   nlp_jac_g  | 226.00us ( 75.33us)  44.85us ( 14.95us)         3
       total  |  16.23ms ( 16.23ms)   2.82ms (  2.82ms)         1
----------- MPC execution -----------
Predicted position of cube [0.33585380871596, 0.29393987861759, 0.084989999769591]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  83.00us ( 27.67us)  83.47us ( 27.82us)         3
       nlp_g  |  34.00us ( 11.33us)  33.32us ( 11.11us)         3
    nlp_grad  | 100.00us (100.00

----------- MPC execution -----------
Predicted position of cube [0.33585380865458, 0.29393987830051, 0.084989999773075]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 136.00us ( 45.33us) 136.52us ( 45.51us)         3
       nlp_g  |  44.00us ( 14.67us)  44.07us ( 14.69us)         3
    nlp_grad  | 164.00us (164.00us) 164.23us (164.23us)         1
  nlp_grad_f  |   7.50ms (  2.50ms) 377.00us (125.67us)         3
  nlp_hess_l  |   1.32ms (659.00us)   1.32ms (659.17us)         2
   nlp_jac_g  |  68.00us ( 22.67us)  67.21us ( 22.40us)         3
       total  |  11.36ms ( 11.36ms)   4.23ms (  4.23ms)         1
----------- MPC execution -----------
Predicted position of cube [0.33585380865458, 0.29393987830051, 0.084989999773075]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 113.00us ( 37.67us) 112.51us ( 37.50us)         3
       nlp_g  |  36.00us ( 12.00us)  35.97us ( 11.99us)         3
    nlp_grad  | 136.00us (136.00

In [10]:
obj.end_simulation()

max(MPC_component.solver_times)

Ending simulation


0.016830921173095703